# Lab 6

Scikit learn provides a large variety of algorithms for some common Machine Learning tasks, such as:

* Classification
* Regression
* Clustering
* Feature Selection
* Anomaly Detection

It also provides some datasets that you can use to test these algorithms:

* Classification Datasets:
    * Breast cancer wisconsin
    * Iris plants (3-classes)
    * Optical recognition of handwritten digits (10-classes)
    * Wine (n-classes)

* Regression Datasets: 
    * Boston house prices 
    * Diabetes
    * Linnerrud (multiple regression)
    * California Housing

* Image: 
    * The Olivetti faces
    * The Labeled Faces in the Wild face recognition
    * Forest covertypes

* NLP:
    * News group
    * Reuters Corpus Volume I 

* Other:
    * Kddcup 99- Intrusion Detection

## Exercises

1. Use the full [Kddcup](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) dataset to compare classification performance of 3 different classifiers. 
    * Separate the data into train, validation, and test. 
    * Use accuracy as the metric for assessing performance. 
    * For each classifier, identify the hyperparameters. Perform optimization over at least 2 hyperparameters.   
    * Compare the performance of the optimal configuration of the classifiers.

2. Pick the best algorithm in question 1. Create an ensemble of at least 25 models, and use them for the classification task. Identify the top and bottom 10% of the data in terms of uncertainty of the decision.

3. Use 2 different feature selection algorithm to identify the 10 most important features for the task in question 1. Retrain classifiers in question 1 with just this subset of features and compare performance.

4. Use the same data, removing the labels, and compare performance of 3 different clustering algorithms. Can you find clusters for each of the classes in question 1? 

5. Can you identify any clusters within the top/botton 10% identified in 2. What are their characteristics?

6. Use the "SA" dataset to compare the performance of 3 different anomaly detection algorithms.

7. Create a subsample of 250 datapoints, redo question 6, using Leave-one-out as the method of evaluation.

8. Use the feature selection algorithm to identify the 5 most important features for the task in question 6, for each algorithm. Does the anomaly detection improve using less features?

## Quick look at the data

In [41]:
from sklearn.datasets import fetch_kddcup99
D=fetch_kddcup99()

In [4]:
dir(D)

['DESCR', 'data', 'feature_names', 'frame', 'target', 'target_names']

In [5]:
print(D["DESCR"])

.. _kddcup99_dataset:

Kddcup 99 dataset
-----------------

The KDD Cup '99 dataset was created by processing the tcpdump portions
of the 1998 DARPA Intrusion Detection System (IDS) Evaluation dataset,
created by MIT Lincoln Lab [2]_. The artificial data (described on the `dataset's
homepage <https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html>`_) was
generated using a closed network and hand-injected attacks to produce a
large number of different types of attack with normal activity in the
background. As the initial goal was to produce a large training set for
supervised learning algorithms, there is a large proportion (80.1%) of
abnormal data which is unrealistic in real world, and inappropriate for
unsupervised anomaly detection which aims at detecting 'abnormal' data, i.e.:

* qualitatively different from normal data
* in large minority among the observations.

We thus transform the KDD Data set into two different data sets: SA and SF.

* SA is obtained by simply selecting all

In [6]:
dir(D)

['DESCR', 'data', 'feature_names', 'frame', 'target', 'target_names']

In [7]:
import numpy as np
np.unique(D["target"])

array([b'back.', b'buffer_overflow.', b'ftp_write.', b'guess_passwd.',
       b'imap.', b'ipsweep.', b'land.', b'loadmodule.', b'multihop.',
       b'neptune.', b'nmap.', b'normal.', b'perl.', b'phf.', b'pod.',
       b'portsweep.', b'rootkit.', b'satan.', b'smurf.', b'spy.',
       b'teardrop.', b'warezclient.', b'warezmaster.'], dtype=object)

In [8]:
len(np.unique(D["target"]))

23

In [9]:
D["feature_names"]

['duration',
 'protocol_type',
 'service',
 'flag',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate']

# Exercise 1

In [80]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from tensorflow.keras.optimizers import Adam
import xgboost as xgb

In [60]:
#defining X
X=pd.DataFrame(data=D['data'],columns=D['feature_names'])

#encoding
en=LabelEncoder()
convert=X.select_dtypes(include=['object']).columns.tolist()

for col in convert:
    le = LabelEncoder()
    X[col] = en.fit_transform(X[col])

X


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,1,22,9,164,4773,0,0,0,0,...,9,9,100,0,11,0,0,0,0,0
1,0,1,22,9,222,465,0,0,0,0,...,19,19,100,0,5,0,0,0,0,0
2,0,1,22,9,218,1316,0,0,0,0,...,29,29,100,0,3,0,0,0,0,0
3,0,1,22,9,202,1316,0,0,0,0,...,39,39,100,0,3,0,0,0,0,0
4,0,1,22,9,200,2006,0,0,0,0,...,49,49,100,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,1,22,9,293,1856,0,0,0,0,...,86,255,100,0,1,5,0,1,0,0
494017,0,1,22,9,265,2254,0,0,0,0,...,6,255,100,0,17,5,0,1,0,0
494018,0,1,22,9,186,1179,0,0,0,0,...,16,255,100,0,6,5,6,1,0,0
494019,0,1,22,9,274,1179,0,0,0,0,...,26,255,100,0,4,5,4,1,0,0


In [61]:
#defining y
y=D['target']

#encoding
encoder=LabelEncoder()
y=encoder.fit_transform(y)
y

array([11, 11, 11, ..., 11, 11, 11])

In [62]:
#checking dataset
df=pd.DataFrame(data=X,columns=D['feature_names'])
df['target']=D['target']
df.describe(include='all')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target
count,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,...,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'smurf.'
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280790
mean,11.936043,0.467132,23.408894,7.842446,595.755899,357.837126,0.000045,0.004469,0.000014,0.028673,...,188.665670,75.377970,3.090573,60.193476,0.639906,17.499390,12.532269,5.811761,5.741167,NaN
std,131.739314,0.575606,13.538332,2.250853,504.011413,1345.396019,0.006673,0.091523,0.005510,0.584815,...,106.040437,41.078098,10.925911,48.130925,3.643362,37.678711,27.043113,23.058951,23.014032,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,0.000000,14.000000,9.000000,39.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,46.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,0.000000,0.000000,14.000000,9.000000,491.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,100.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
75%,0.000000,1.000000,42.000000,9.000000,988.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,100.000000,4.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [63]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=1) #train = 70%
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1) #test = 15%, validation = 15%
print("X_train.shape", X_train.shape, "y_train.shape", y_train.shape)
print("X_test.shape", X_test.shape, "y_test.shape", y_test.shape)
print("X_val.shape", X_test.shape, "y_val.shape", y_test.shape)

X_train.shape (345814, 41) y_train.shape (345814,)
X_test.shape (74104, 41) y_test.shape (74104,)
X_val.shape (74104, 41) y_val.shape (74104,)


In [64]:
#standardizing
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [68]:
#function to create a NN model
def create_nn(reg=tf.keras.regularizers.l2(0.1), learning_rate=0.001):
    model=keras.Sequential(
        [
            Input(shape=(X_train.shape[1],)),
            Dense(100, activation='relu',kernel_regularizer=reg),
            Dense(50, activation='relu',kernel_regularizer=reg),
            Dense(len(np.unique(y_train)), activation='softmax')
        ]
    )
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

#hyperparameter tuning for NN
nn_results=[]
for reg in [tf.keras.regularizers.l1(0.1),tf.keras.regularizers.l2(0.1)]: #hyperparameter 1: regularization
    for learning_rate in [0.001,0.01]: #hyperparameter 2: learning rate
        model = create_nn(reg, learning_rate)
        model.fit(X_train, y_train, epochs=10, verbose=0)
        loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
        nn_results.append((reg, learning_rate, accuracy))

#finding best model (highest accuracy)
best_nn=max(nn_results, key=lambda x: x[2])
best_nn

(<keras.src.regularizers.regularizers.L2 at 0x34ca12cf0>,
 0.001,
 0.9925779104232788)

In [82]:
#svm 
svm=SVC()
svm.fit(X_train, y_train)
param_grid={'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['linear']}  

grid=GridSearchCV(SVC(), param_grid, refit=True, scoring='accuracy', verbose=3) 
grid.fit(X_train, y_train) 
svm_best_params=grid.best_params_
print("Best Hyperparameters:", svm_best_params)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.999 total time=  11.9s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.999 total time=  10.7s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.999 total time=  11.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.999 total time=  11.5s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.999 total time=  11.2s
[CV 1/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.999 total time=  10.9s
[CV 2/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.999 total time=  10.4s
[CV 3/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.999 total time=  11.0s
[CV 4/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.999 total time=  10.8s
[CV 5/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.999 total time=  11.1s
[CV 1/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.999 total time=  10.8s
[CV 2/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.999 total time=  10.4s
[CV 3/5] END ..C=0.1, gamma=

In [83]:
#svm metrics
#Obtaining the best performing model from the grid search results
svm_best_model=grid.best_estimator_

#prediction
y_pred_svm=svm_best_model.predict(X_test)

#metrics
accuracy_svm=accuracy_score(y_test, y_pred_svm)
accuracy_svm

0.9994737126201014

In [74]:
#XGBoost
xgb_model=xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)
param_grid={
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.3]
}
grid_search=GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)
grid_search.fit(X_train, y_train)
best_params=grid_search.best_params_
print("Best Hyperparameters:", best_params)


Fitting 3 folds for each of 27 candidates, totalling 81 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   4.4s
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   4.4s
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   4.5s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   8.9s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   9.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   8.6s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=  17.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=  16.7s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=  18.4s
[CV] END ...learning_rate=0.01, max_depth=4, n_estimators=50; total time=   4.4s
[CV] END ...learning_rate=0.01, max_depth=4, n_estimators=50; total time=   4.6s
[CV] END ...learning_rate=0.01, max_depth=4, n_estimators=50; total time=   4.7s
[CV] END ..learning_rate=0.0

In [76]:
#XGBoost metrics
#Obtaining the best performing model from the grid search results
xgb_best_model=grid_search.best_estimator_

#prediction
y_pred_xgb=xgb_best_model.predict(X_test)

#metrics
accuracy_xgb=accuracy_score(y_test, y_pred_xgb)
accuracy_xgb

0.9996356471985318

In [86]:
# Compile results
results = {
    'Neural Network': best_nn,
    'SVM': (svm_best_params, accuracy_svm),
    'XGBoost': (best_params, accuracy_xgb)
}

# Print results
print('Accuracy Results:')
for model, result in results.items():
    if model == 'Neural Network':
        print(f'{model}: accuracy={result[2]:.4f}, regularization={result[0]}, learning_rate={result[1]}')
    elif model == 'K-Means':
        print(f'{model}: accuracy={result[1]:.4f}, n_clusters={result[0]}')
    else:
        print(f'{model}: accuracy={result[1]:.4f}, params={result[0]}')

Accuracy Results:
Neural Network: accuracy=0.9926, regularization=<keras.src.regularizers.regularizers.L2 object at 0x34ca12cf0>, learning_rate=0.001
SVM: accuracy=0.9995, params={'C': 1, 'gamma': 1, 'kernel': 'linear'}
XGBoost: accuracy=0.9996, params={'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 200}


# Exercise 2

In [91]:
from sklearn.ensemble import BaggingClassifier

#ensemble using XGBoost (best classifier from ex1)
ensemble_model=BaggingClassifier(xgb_best_model, n_estimators=25)
ensemble_model.fit(X_train, y_train)

#predictions and uncertainty
predictions=ensemble_model.predict(X_test)
probs=ensemble_model.predict_proba(X_test)
uncertainty=np.max(probs, axis=1)

#top/bottom percent of uncertainty (thresholds)
top10=np.percentile(uncertainty, 90)
bottom10=np.percentile(uncertainty, 10)

top10_percent=X_test[uncertainty>=top10]
bottom10_percent=X_test[uncertainty<=bottom10]

print(f'Top 10%: \n{top10_percent}')
print(f'Bottom 10%: \n{bottom10_percent}')

Top 10%: 
[[-0.0908001  -0.81154971 -0.69475967 ... -0.46343761 -0.25219548
  -0.24957278]
 [-0.0908001  -0.81154971 -0.69475967 ... -0.46343761 -0.25219548
  -0.24957278]
 [-0.0908001  -0.81154971 -0.69475967 ... -0.46343761 -0.25219548
  -0.24957278]
 ...
 [-0.0908001  -0.81154971 -0.69475967 ... -0.46343761 -0.25219548
  -0.24957278]
 [-0.0908001  -0.81154971 -0.69475967 ... -0.46343761 -0.25219548
  -0.24957278]
 [-0.0908001  -0.81154971 -0.69475967 ... -0.46343761 -0.25219548
  -0.24957278]]
Bottom 10%: 
[[-0.0908001   0.92425714  1.59547251 ...  2.16188055 -0.25219548
  -0.24957278]
 [-0.0908001   0.92425714 -0.4731243  ...  0.05423076 -0.25219548
  -0.24957278]
 [-0.0908001   0.92425714 -0.10373201 ... -0.46343761  3.34593975
   4.09498457]
 ...
 [-0.0908001   0.92425714  1.59547251 ... -0.46343761  4.08290721
   4.09498457]
 [-0.0908001   0.92425714  1.59547251 ... -0.46343761  4.08290721
   4.09498457]
 [-0.0908001   0.92425714 -0.4731243  ... -0.46343761 -0.25219548
  -0.2495

# Exercise 3

In [95]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

#rfe
rf=RandomForestClassifier()
rfeselector=RFE(rf, n_features_to_select=10) #10 most important features
rfeselector=rfeselector.fit(X_train, y_train)
X_train=pd.DataFrame(X_train)
rfeselect=X_train.columns[rfeselector.support_]

#feature importance
rf.fit(X_train, y_train)
importances=rf.feature_importances_
indices=np.argsort(importances)[-10:] #10 most important features
importanceselect=X_train.columns[indices]

#redefining X_train to only include the selected features
X_rfe=X_train[rfeselect]
X_importance=X_train[importanceselect]


In [96]:
#redefining X_test and X_val
X_test=pd.DataFrame(X_test)
X_val=pd.DataFrame(X_val)

rfe_Xtest_sel=X_test.columns[rfeselector.support_]
rfe_Xval_sel=X_val.columns[rfeselector.support_]

fi_Xtest_sel=X_test.columns[indices]
fi_Xval_sel=X_val.columns[indices]

rfe_Xtest=X_test[rfe_Xtest_sel]
rfe_Xval=X_val[rfe_Xval_sel]

fi_Xtest=X_test[fi_Xtest_sel]
fi_Xval=X_val[fi_Xval_sel]

#### RFE Selections: Retraining classifiers from Exercise 1

In [98]:
#hyperparameter tuning for NN
#function to create a NN model
def create_nn_rfe(reg=tf.keras.regularizers.l2(0.1), learning_rate=0.001):
    model=keras.Sequential(
        [
            Input(shape=(X_rfe.shape[1],)),
            Dense(100, activation='relu',kernel_regularizer=reg),
            Dense(50, activation='relu',kernel_regularizer=reg),
            Dense(len(np.unique(y_train)), activation='softmax')
        ]
    )
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

rfe_nn_results=[]
for reg in [tf.keras.regularizers.l1(0.1),tf.keras.regularizers.l2(0.1)]: #hyperparameter 1: regularization
    for learning_rate in [0.001,0.01]: #hyperparameter 2: learning rate
        model = create_nn_rfe(reg, learning_rate)
        model.fit(X_rfe, y_train, epochs=10, verbose=0)
        loss, accuracy = model.evaluate(rfe_Xval, y_val, verbose=0)
        rfe_nn_results.append((reg, learning_rate, accuracy))

#finding best model (highest accuracy)
rfe_best_nn=max(rfe_nn_results, key=lambda x: x[2])
rfe_best_nn

(<keras.src.regularizers.regularizers.L2 at 0x3519c8950>,
 0.001,
 0.9873149394989014)

In [99]:
#svm 
svm=SVC()
svm.fit(X_rfe, y_train)
param_grid={'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['linear']}  

rfe_grid=GridSearchCV(SVC(), param_grid, refit=True, scoring='accuracy', verbose=3) 
rfe_grid.fit(X_rfe, y_train) 
rfe_svm_best_params=rfe_grid.best_params_
print("Best Hyperparameters:", rfe_svm_best_params)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.996 total time=  10.5s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.996 total time=  10.4s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.996 total time=  10.4s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.996 total time=  10.6s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.996 total time=  10.5s
[CV 1/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.996 total time=  10.6s
[CV 2/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.996 total time=  10.3s
[CV 3/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.996 total time=  10.5s
[CV 4/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.996 total time=  10.7s
[CV 5/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.996 total time=  10.5s
[CV 1/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.996 total time=  10.6s
[CV 2/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.996 total time=  10.4s
[CV 3/5] END ..C=0.1, gamma=

In [100]:
#svm metrics
#Obtaining the best performing model from the grid search results
rfe_svm_best_model=rfe_grid.best_estimator_

#prediction
rfe_y_pred_svm=rfe_svm_best_model.predict(rfe_Xtest)

#metrics
rfe_accuracy_svm=accuracy_score(y_test, rfe_y_pred_svm)
rfe_accuracy_svm

0.9969772212026341

In [101]:
#XGBoost
xgb_model=xgb.XGBClassifier()
xgb_model.fit(X_rfe, y_train)
param_grid={
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.3]
}
rfe_grid_search=GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)
rfe_grid_search.fit(X_rfe, y_train)
rfe_best_params=rfe_grid_search.best_params_
print("Best Hyperparameters:", rfe_best_params)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   3.2s
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   3.2s
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   3.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   6.2s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   6.2s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   6.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=  12.5s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=  12.9s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=  13.0s
[CV] END ...learning_rate=0.01, max_depth=4, n_estimators=50; total time=   3.2s
[CV] END ...learning_rate=0.01, max_depth=4, n_estimators=50; total time=   3.2s
[CV] END ...learning_rate=0.01, max_depth=4, n_estimators=50; total time=   3.3s
[CV] END ..learning_rate=0.0

In [102]:
#XGBoost metrics
#Obtaining the best performing model from the grid search results
rfe_xgb_best_model=rfe_grid_search.best_estimator_

#prediction
rfe_y_pred_xgb=rfe_xgb_best_model.predict(rfe_Xtest)

#metrics
rfe_accuracy_xgb=accuracy_score(y_test, rfe_y_pred_xgb)
rfe_accuracy_xgb

0.9992982834934686

#### Feature Importance Selections: Retraining classifiers from Exercise 1

In [104]:
#hyperparameter tuning for NN
#function to create a NN model
def create_nn_fi(reg=tf.keras.regularizers.l2(0.1), learning_rate=0.001):
    model=keras.Sequential(
        [
            Input(shape=(X_importance.shape[1],)),
            Dense(100, activation='relu',kernel_regularizer=reg),
            Dense(50, activation='relu',kernel_regularizer=reg),
            Dense(len(np.unique(y_train)), activation='softmax')
        ]
    )
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

fi_nn_results=[]
for reg in [tf.keras.regularizers.l1(0.1),tf.keras.regularizers.l2(0.1)]: #hyperparameter 1: regularization
    for learning_rate in [0.001,0.01]: #hyperparameter 2: learning rate
        model = create_nn_fi(reg, learning_rate)
        model.fit(X_importance, y_train, epochs=10, verbose=0)
        loss, accuracy = model.evaluate(fi_Xval, y_val, verbose=0)
        fi_nn_results.append((reg, learning_rate, accuracy))

#finding best model (highest accuracy)
fi_best_nn=max(fi_nn_results, key=lambda x: x[2])
fi_best_nn

(<keras.src.regularizers.regularizers.L2 at 0x352ae43e0>,
 0.001,
 0.9865997433662415)

In [105]:
#svm 
svm=SVC()
svm.fit(X_importance, y_train)
param_grid={'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['linear']}  

fi_grid=GridSearchCV(SVC(), param_grid, refit=True, scoring='accuracy', verbose=3) 
fi_grid.fit(X_importance, y_train) 
fi_svm_best_params=fi_grid.best_params_
print("Best Hyperparameters:", fi_svm_best_params)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.996 total time=  10.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.996 total time=   9.9s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.996 total time=  10.2s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.996 total time=  10.5s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.996 total time=  10.4s
[CV 1/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.996 total time=  10.5s
[CV 2/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.996 total time=  10.3s
[CV 3/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.996 total time=  10.5s
[CV 4/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.996 total time=  10.7s
[CV 5/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.996 total time=  10.6s
[CV 1/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.996 total time=  10.6s
[CV 2/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.996 total time=  10.4s
[CV 3/5] END ..C=0.1, gamma=

In [106]:
#svm metrics
#Obtaining the best performing model from the grid search results
fi_svm_best_model=fi_grid.best_estimator_

#prediction
fi_y_pred_svm=fi_svm_best_model.predict(fi_Xtest)

#metrics
fi_accuracy_svm=accuracy_score(y_test, fi_y_pred_svm)
fi_accuracy_svm

0.9969772212026341

In [107]:
#XGBoost
xgb_model=xgb.XGBClassifier()
xgb_model.fit(X_importance, y_train)
param_grid={
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.3]
}
fi_grid_search=GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)
fi_grid_search.fit(X_rfe, y_train)
fi_best_params=fi_grid_search.best_params_
print("Best Hyperparameters:", fi_best_params)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   3.0s
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   3.2s
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   3.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   6.3s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   5.8s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   5.7s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=  11.5s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=  11.9s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=  12.5s
[CV] END ...learning_rate=0.01, max_depth=4, n_estimators=50; total time=   3.2s
[CV] END ...learning_rate=0.01, max_depth=4, n_estimators=50; total time=   3.0s
[CV] END ...learning_rate=0.01, max_depth=4, n_estimators=50; total time=   3.1s
[CV] END ..learning_rate=0.0

In [108]:
#XGBoost metrics
#Obtaining the best performing model from the grid search results
fi_xgb_best_model=fi_grid_search.best_estimator_

#prediction
fi_y_pred_xgb=fi_xgb_best_model.predict(fi_Xtest)

#metrics
fi_accuracy_xgb=accuracy_score(y_test, fi_y_pred_xgb)
fi_accuracy_xgb

ValueError: feature_names mismatch: ['1', '2', '3', '4', '22', '23', '28', '29', '33', '35'] ['3', '2', '29', '33', '4', '28', '1', '35', '23', '22']

In [113]:
#Comparison for ex3
comparison=pd.DataFrame({'Accuracy':['Original','RFE','Feature Importance'],
                       'NN':[best_nn[2],rfe_best_nn[2],fi_best_nn[2]],
                       'SVM':[accuracy_svm,rfe_accuracy_svm,fi_accuracy_svm],
                       'XGB':[accuracy_xgb,rfe_accuracy_xgb,'']})#fi_accuracy_xgb]})
comparison

,Accuracy,NN,SVM,XGB
0,Original,0.992578,0.999474,0.999636
1,RFE,0.987315,0.996977,0.999298
2,Feature Importance,0.986600,0.996977,


# Exercise 4

# Exercise 5

# Exercise 6

# Exercsie 7

# Exercise 8